##### INITIAL CONFIGURATION

In [1]:
# USED LIBRARIES
import pandas as pd
from datetime import datetime
from datetime import timedelta
pd.options.mode.chained_assignment = None

In [2]:
# READ FILES
trips = pd.read_csv(r'..\data_in\T\ES\trips.txt', sep=",") # viajes distintos que hay
calendar = pd.read_csv(r'..\data_in\T\ES\calendar.txt', sep=",") # que dias ocurre el tipo de viaje
calendar_dates = pd.read_csv(r'..\data_in\T\ES\calendar_dates.txt', sep=",") # excepciones a la tabla calendar
stop_times = pd.read_csv(r'..\data_in\T\ES\stop_times.txt', sep=",") # paradas de cada viaje
stations = pd.read_csv(r'..\data_in\T\ES\stops.txt', sep=",") # estaciones

##### TRANSFORM INITIAL TABLES

In [3]:
# REMOVING NOT USED COLUMNS
calendar = calendar[["service_id", "start_date", calendar.columns[9]]]
stop_times = stop_times[["trip_id", "stop_id", "stop_sequence"]]
stations = stations[["stop_id", "stop_name", "stop_lat", "stop_lon"]]
calendar_dates.drop(columns = calendar_dates.columns[2], inplace = True)

In [4]:
# RENAME COLUMNS
column_names = {'stop_id':'station_id',
    "stop_name":"station_name",
    "stop_lat":"station_latitud",
    "stop_lon":"station_longitud"}
stations.rename(columns = column_names, inplace = True)

In [5]:
# CHANGE COLUMNS DATA TYPE TO DATETIME
calendar['first_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['last_date'] = pd.to_datetime(calendar[calendar.columns[2]], format='%Y%m%d')
calendar_dates['date'] = pd.to_datetime(calendar_dates['date'], format='%Y%m%d')

In [6]:
# COLUMN WITH DAYS BETWEEN START AND END OF DATES
calendar['days_diff'] = calendar.apply(lambda x: (x['last_date'] - x['first_date']).days, axis=1)

##### FIND ORIGIN AND DESTINATION STATIONS OF EACH TRIP

In [7]:
# CREATES AN INVERSE OF THE STOP SEQUENCE TO SEE THE FINAL STOP FOR EACH TRIP
# works as ROW_NUMBER() OVER (PARTITION BY trip_id ORDER BY stop_sequence DESC)
stop_times['stop_sequence_inv'] = stop_times.groupby("trip_id")["stop_sequence"].rank(method="first", ascending=False)
stop_times["stop_sequence_inv"] = stop_times["stop_sequence_inv"].astype(int)

In [8]:
# DF WITH ONLY THE INITIAL STOP
first_stop = stop_times.loc[stop_times['stop_sequence'] == 1]

# DF WITH ONLY THE LAST STOP
last_stop = stop_times.loc[stop_times['stop_sequence_inv'] == 1]

In [9]:
# RENAME COLUMNS
column_names = {"stop_id":"station_start_id"}
first_stop.rename(columns = column_names, inplace = True)

column_names = {"stop_id":"station_end_id"}
last_stop.rename(columns = column_names, inplace = True)

In [10]:
# REMOVE COLUMNS
first_stop = first_stop[["trip_id", "station_start_id"]]
last_stop = last_stop[["trip_id", "station_end_id"]]

In [11]:
# JOIN THE FIRST AND LAST STOP TO THE TRIPS TABLE
trips = trips.merge(first_stop, on='trip_id', how='left')
trips = trips.merge(last_stop, on='trip_id', how='left')

In [12]:
# REMOVE COLUMNS
trips = trips[["service_id", "station_start_id", "station_end_id"]]

##### CREATE FACT TABLE

In [13]:
# EMPTY LISTS
service_id_list = []
dates_list = []

In [14]:
# ITERATE CALENDAR TABLES TO OBTAIN THE DATA
for index, row in calendar.iterrows():
    for i in range(0, row['days_diff'] + 1):
        service_id_list.append(row['service_id'])
        dates_list.append(row['first_date'] + timedelta(days=10))
    

In [15]:
# CREATE DATAFRAME WITH THE DATA
dict_table = {'service_id': service_id_list, 'date': dates_list}
fact_table = pd.DataFrame(dict_table)

In [16]:
# COLUMN IN CALENDAR_DATES TO DIFFERENTIATE WHEN MERGING
calendar_dates['exception'] = 1

In [17]:
fact_table = fact_table.merge(calendar_dates, on=['service_id', 'date'], how='left')

In [18]:
## REMOVE ROWS WITH AN EXCEPTION
fact_table = fact_table.loc[fact_table['exception'] != 1]

# REMOVE COLUMN
fact_table.drop(columns = ['exception'], inplace = True)

In [19]:
# MERGE TRIPS TABLE TO ADD ORIGIN AND DESTINATION STATIONS
fact_table = fact_table.merge(trips, on=['service_id'], how='left')

##### SAVE FILES

In [20]:
fact_table.to_csv(r'..\data_out\trips_T_ES.csv', sep=";", decimal=',',  index = False)
stations.to_csv(r'..\data_out\stations_T_ES.csv', sep=";", decimal=',', index = False)